In [3]:
import pandas as pd
import numpy as np

In [4]:
playtimegenre = pd.read_csv('PlayTimeGenre.csv')
userforgenre = pd.read_csv('UserForGenre.csv')
usersrecommend = pd.read_csv('UsersRecommend.csv')
usersworstdeveloper = pd.read_csv('UsersWorstDeveloper.csv')
sentimentanalysis = pd.read_csv('sentiment_analysis.csv')
recomendacionjuego = pd.read_csv('product.csv')

In [4]:
def PlayTimeGenre(genero: str):
    # Filtrar el DataFrame por el género especificado
    filtered_df = playtimegenre[playtimegenre['genres'] == genero]

    # Verificar si hay datos para el género especificado
    if filtered_df.empty:
        return {"No hay datos disponibles para el género {}".format(genero)}

    # Agrupar por año y sumar las horas jugadas
    grouped_df = filtered_df.groupby('release_year')['playtime_forever'].sum()

    # Verificar si hay datos después de agrupar
    if grouped_df.empty:
        return {"No hay datos disponibles para el género {}".format(genero)}

    # Encontrar el año con más horas jugadas
    max_year = int(grouped_df.idxmax())  # Convertir el año a entero

    # Construir el diccionario de resultado
    result = {"Año de lanzamiento con más horas jugadas para Género {}".format(genero): max_year}

    return result

In [5]:
PlayTimeGenre('Adventure')

{'Año de lanzamiento con más horas jugadas para Género Adventure': 2011}

In [6]:
def UserForGenre(genero: str):
    # Filtrar el DataFrame por el género dado
    genre_df = userforgenre[userforgenre['genres'] == genero]

    # Agrupar por usuario y sumar las horas jugadas
    user_hours = genre_df.groupby('user_id')['playtime_forever'].sum()

    # Obtener el usuario con más horas jugadas
    max_user = user_hours.idxmax()

    # Filtrar el DataFrame por el usuario con más horas jugadas
    max_user_df = genre_df[genre_df['user_id'] == max_user]

    # Agrupar por año y sumar las horas jugadas
    hours_by_year = max_user_df.groupby('release_year')['playtime_forever'].sum().reset_index()

    # Crear la lista de horas jugadas por año en el formato deseado
    hours_list = [{'Año': year, 'Horas': hours} for year, hours in zip(hours_by_year['release_year'], hours_by_year['playtime_forever'])]

    # Crear el diccionario de retorno
    result = {
        "Usuario con más horas jugadas para Género {}".format(genero): max_user,
        "Horas jugadas": hours_list
    }

    return result

In [7]:
UserForGenre('Action')

{'Usuario con más horas jugadas para Género Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1995, 'Horas': 3},
  {'Año': 2000, 'Horas': 1177},
  {'Año': 2002, 'Horas': 3},
  {'Año': 2003, 'Horas': 127},
  {'Año': 2004, 'Horas': 2123},
  {'Año': 2005, 'Horas': 355},
  {'Año': 2006, 'Horas': 14},
  {'Año': 2007, 'Horas': 1879},
  {'Año': 2008, 'Horas': 3},
  {'Año': 2009, 'Horas': 1805},
  {'Año': 2010, 'Horas': 1301},
  {'Año': 2011, 'Horas': 2581},
  {'Año': 2012, 'Horas': 6304},
  {'Año': 2013, 'Horas': 2007},
  {'Año': 2014, 'Horas': 2178},
  {'Año': 2015, 'Horas': 5125},
  {'Año': 2016, 'Horas': 492},
  {'Año': 2017, 'Horas': 722}]}

In [8]:
def UsersRecommend(año: int):
    # Filtrar el DataFrame para el año dado y donde las recomendaciones sean positivas o neutrales
    filtered_df = usersrecommend[(usersrecommend['year'] == año) & (usersrecommend['recommend_score'] > 0)]

    # Ordenar el DataFrame por el número de recomendaciones de forma descendente
    sorted_df = filtered_df.sort_values(by='recommend_score', ascending=False)

    # Tomar los primeros 3 juegos del DataFrame ordenado
    top_3 = sorted_df.head(3)

    # Crear la lista de resultados en el formato deseado
    result = [{"Puesto 1": top_3.iloc[0]['app_name']},
              {"Puesto 2": top_3.iloc[1]['app_name']},
              {"Puesto 3": top_3.iloc[2]['app_name']}]

    return result

In [11]:
UsersRecommend(2015)

[{'Puesto 1': 'Counter-Strike: Global Offensive'},
 {'Puesto 2': 'Team Fortress 2'},
 {'Puesto 3': "Garry's Mod"}]

In [14]:
def UsersWorstDeveloper(año: int):
    # Filtrar el DataFrame para el año dado y comentarios negativos
    filtered_df = usersworstdeveloper[(usersworstdeveloper['year'] == año) & (usersworstdeveloper['bad_reviews'] > 0)]

    # Agrupar por desarrollador y calcular la suma de malas reseñas y el mínimo de recommend_score
    developer_stats = filtered_df.groupby('developer').agg({'bad_reviews': 'sum', 'recommend_score': ['min', 'sum']})

    # Ordenar primero por bad_reviews en orden descendente y luego por recommend_score y recommend_score_total en orden ascendente
    sorted_developers = developer_stats.sort_values(by=[('bad_reviews', 'sum'), ('recommend_score', 'min'), ('recommend_score', 'sum')],
                                                    ascending=[False, True, True])

    # Tomar las primeras 3 desarrolladoras
    top_developers = sorted_developers.head(3)

    # Crear la lista de resultados en el formato deseado
    result = [{"Puesto {}".format(i + 1): developer}
              for i, developer in enumerate(top_developers.index)]

    return result

In [20]:
UsersWorstDeveloper(2014)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Bohemia Interactive'},
 {'Puesto 3': 'Facepunch Studios'}]

In [24]:
def sentiment_analysis(desarrolladora: str):
    # Filtrar el DataFrame por la empresa desarrolladora
    empresa_df = sentimentanalysis[sentimentanalysis['developer'] == desarrolladora]

    # Calcular la cantidad total de cada categoría de sentimiento
    total_negativas = int(empresa_df['bad_reviews'].sum())
    total_neutrales = int(empresa_df['neutral_reviews'].sum())
    total_positivas = int(empresa_df['good_reviews'].sum())

    # Crear el diccionario de retorno
    resultado = {
        desarrolladora: {
            'Negative': total_negativas,
            'Neutral': total_neutrales,
            'Positive': total_positivas
        }
    }

    return resultado

In [25]:
sentiment_analysis('高考恋爱委员会,橘子班')

{'高考恋爱委员会,橘子班': {'Negative': 0, 'Neutral': 1, 'Positive': 1}}

In [23]:
def recomendacion_juego(game_id: int):

    path = 'cosine_sim.npy'
    cosine_sim = np.load(path)

    idx = recomendacionjuego[recomendacionjuego['id'] == game_id].index[0]
    rec_games = recomendacionjuego['app_name'].iloc[cosine_sim[idx]]

    info = {'recomendaciones': None}
    info['recomendaciones'] = list(rec_games)

    return info

In [25]:
recomendacion_juego(10)

{'recomendaciones': ['Half-Life Deathmatch: Source',
  'Zombie Panic',
  'Cannon Fodder 3',
  'Counter-Strike: Condition Zero',
  'Zombie Master']}